Importando Bibliotecas

In [57]:
import pandas as pd

Lendo Base de Dados

In [58]:
controle_path = 'bases'
def ler_arquivo (nome_arquivo):
    try:
        return pd.read_excel(f'{controle_path}/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'

In [59]:
nome_arquivo = 'dados de entregas'
df = ler_arquivo(nome_arquivo)

In [60]:
df

,id_entrega,filial,cliente,status,descricao_entrega,tipo_servico,valor_frete,data_postagem,prazo_previsto,data_prevista,data_entrega,dias_atraso
0,3619,SC,Americanas,ENTREGUE,Extravio temporário,ECONÔMICO,25.16,2026-01-09 20:43:34.883,5,2026-01-14 20:43:34.883,2026-01-15 20:43:34.883,1
1,4850,RJ,Amazon,ENTREGUE,Entrega reagendada,NORMAL,38.17,2026-01-06 20:43:34.889,5,2026-01-11 20:43:34.889,2026-01-10 20:43:34.889,0
2,3232,MG,Magazine Luiza,EM TRÂNSITO,Cliente ausente no local,NORMAL,311.62,2026-02-03 20:43:34.881,1,2026-02-04 20:43:34.881,2026-02-05 20:43:34.881,1
3,3944,RS,Amazon,EM TRÂNSITO,Cliente ausente no local,NORMAL,65.46,2026-01-17 20:43:34.885,7,2026-01-24 20:43:34.885,2026-01-26 20:43:34.885,2
4,2633,SP,Amazon,EM TRÂNSITO,Problema operacional na rota,ECONÔMICO,104.62,2026-02-03 20:43:34.875,2,2026-02-05 20:43:34.875,2026-02-05 20:43:34.875,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5195,3997,SC,Shopee,ENTREGUE,Entrega normal,NORMAL,38.12,2026-01-23 20:43:34.885,7,2026-01-30 20:43:34.885,2026-01-31 20:43:34.885,1
5196,3250,PR,Mercado Livre,ENTREGUE,Problema operacional na rota,ECONÔMICO,69.88,2026-01-19 20:43:34.881,2,2026-01-21 20:43:34.881,2026-01-19 20:43:34.881,0
5197,2834,PR,Magazine Luiza,EM TRÂNSITO,Extravio temporário,ECONÔMICO,152.43,2026-01-15 20:43:34.877,7,2026-01-22 20:43:34.877,2026-01-24 20:43:34.877,2
5198,507,MG,Amazon,EM TRÂNSITO,Entrega EXPRESSA,ECONÔMICO,27.43,2026-02-03 20:43:34.864,2,2026-02-05 20:43:34.864,2026-02-07 20:43:34.864,2


SLA

In [63]:
# Total de entregas sem atraso
sla_ok = df.loc[(df['dias_atraso'] == 0) & (df['status']=='ENTREGUE')].shape[0]
display(sla_ok)

# Total de entregas
total_entregas = df.loc[(df['status'] == 'ENTREGUE')].shape[0]
# display(total_entregas)

# Percentual de SLA
sla_percentual = (sla_ok/total_entregas)*100

f'{round(sla_percentual,2)}% de entregas dentro do prazo'

932

'37.24% de entregas dentro do prazo'

KPI - Entregas atrasadas

In [64]:
# Entregas atrasadas

sla_atrasadas = df.loc[(df['dias_atraso']>0) 
    & (df['status'] == 'ENTREGUE')
].shape[0]

atraso_percentual = (sla_atrasadas/total_entregas)*100

f'{round(atraso_percentual,2)}% de entregas atrasadas'

'62.76% de entregas atrasadas'

SLA - Identificando onde as operações funcionam melhor

In [75]:
sla_final = (
    df.loc[(df['status']=='ENTREGUE')]
).groupby('filial')['dias_atraso'].mean()

round(sla_final,2)

filial
MG    1.84
PR    1.92
RJ    1.81
RS    1.97
SC    1.91
SP    1.80
Name: dias_atraso, dtype: float64

In [84]:
sla_final = (
    df.loc[(df['status']=='ENTREGUE')]
).groupby('filial')['dias_atraso'].apply(
    lambda grupo: (grupo == 0).mean()*100
).sort_values(ascending=False)

round(sla_final,2).reset_index(name="% Entregas no prazo")

,filial,% Entregas no prazo
0,RJ,42.08
1,PR,38.34
2,MG,38.25
3,SP,35.76
4,RS,34.93
5,SC,34.76
